In [1]:
import glob
import json
import dask.bag as db

In [2]:
def parse_text(data_list):
    # Clean and format the text for conversion into a dictionary
    
    data_parsed_list = [i.strip() for i in data_list] # Parse text
    data_parsed_strip_list = list(filter(None, data_parsed_list)) # Remove empty lines
    
    # Join lines that don't have a header
    i = 1
    while i < len(data_parsed_strip_list):
        if '<B>' not in data_parsed_strip_list[i]:
            data_parsed_strip_list[i-1:i+1] = [' '.join(data_parsed_strip_list[i-1:i+1])]
        else:
            i+=1
    
    data_dict_list = [item.replace('<B>', '') for item in data_parsed_strip_list] # Remove html tags
    
    return data_dict_list

def list_to_dict(data_dict_list):
    # Takes data_list and converts it into a dictionary
    
    data_dict = {}
    for i in range(len(data_dict_list)):
        if '</B>' not in data_dict_list[i]:
            data_dict['text_{}'.format(i)] = data_dict_list[i]
        else:
            pair = data_dict_list[i].split('</B>', 1)
            data_dict[pair[0].replace(':', '').strip()] = pair[1].strip()
        
    return data_dict

In [3]:
files_list = glob.glob("data/data/*.txt") # Grab a list of filenames
files_list = sorted(files_list)

In [6]:
# Reads files into memory

text_json = []

for i in files_list:
    with open(i, 'r', errors='replace') as f:
        text = f.readlines()
        f.close
    
    text_list = parse_text(text) # Parse the text into a list of key-value pairs in string format
    text_dict = list_to_dict(text_list) # Converts the key-value pairs into dictionary
    text_json.append(text_dict) # Appends the dictionary to a list to create json

In [7]:
# Save file as actual JSON
filename = "medical-text-data"
with open('data/{}.json'.format(filename), 'w') as f:
    for i in range(len(text_json)):
        json.dump(text_json[i], f)
        f.write('\n')
print("Complete")

Complete


In [8]:
text_full_bag = db.read_text("data/medical-text-data.json".format(files_list[0])).map(json.loads) # Loads the json file as a dask bag
text_tuple = text_full_bag.take(10) # Takes the first 10000 entries of the dask bag and stores as a tuple
text_tuple